# Converts Data into Format for AMPL .dat 

In [26]:
import pandas as pd
import numpy as np

In [27]:
#Not required for the generation of output text file
from tqdm import tqdm_notebook

## Step 1: Resource Parameter

* Money - the total amount of money that can be spent all counties
* Increment - the level of precision that the model will consider, ex: increment = 100 means the money will spend either 0,100,200,300,... per county
* Limit - Max that the model can spend on a county. Increment should evenly divide this number. Can be tuned later to adjust for computation time without changing optimal value

In [28]:
money = 100000000
increment = 10000
limit = 500000

## Step 2: States & Entry Cost

Read in CSV File with col1 as the states and col2 as the associated entry cost for that state

In [29]:
state = pd.read_csv("Text Files/StatesCost.csv")

In [30]:
#state.head()

## Step 3: Counties & Characteristics

Read in CSV File with county characteristics

In [31]:
county = pd.read_csv("Text Files/CountyChar.csv")

In [32]:
#county.head()

## Step 4: Generate Pairs Parameter

In [33]:
pairs = pd.DataFrame(county[county.columns[0]])

In [34]:
#function to check if state is appended to the end of county
def checkin(s_name, county):
    if "," + s_name in county:
        return 1
    else:
        return 0

In [35]:
#removes commas
def namefix(county):
    return county.replace(',','')

In [36]:
#removes spaces
def spacefix(county):
    return county.replace(' ','')

In [37]:
#removes '
def apfix(county):
    return county.replace("'", '')

In [41]:
def findstate(county):
    i = county.find(',')
    return county[i+1:]

In [44]:
#Used later in baseline calculation
baseline = pd.DataFrame()
baseline['County'] = county[county.columns[0]]
baseline['State'] = baseline['County'].apply(findstate)

Can be rewritten for conciseness and usability

In [13]:
#creates the pairing using the above functions
pairs[pairs.columns[0]] = pairs[pairs.columns[0]].apply(apfix)
county[county.columns[0]] = county[county.columns[0]].apply(apfix)
pairs[pairs.columns[0]] = pairs[pairs.columns[0]].apply(spacefix)
county[county.columns[0]] = county[county.columns[0]].apply(spacefix)
for state_name in state[state.columns[0]]:
    pairs[state_name] = np.vectorize(checkin)(state_name, county[county.columns[0]])
pairs[pairs.columns[0]] = pairs[pairs.columns[0]].apply(namefix)
county[county.columns[0]] = county[county.columns[0]].apply(namefix)

In [14]:
pairs.head()

,County,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,DistrictofColumbia,...,SouthDakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,WestVirginia,Wisconsin,Wyoming
0,AutaugaCountyAlabama,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BaldwinCountyAlabama,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BarbourCountyAlabama,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BibbCountyAlabama,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BlountCountyAlabama,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 5: Run This

In [15]:
#Uncomment an run this cell if the first time using this file or if you want the output file to have a different name
### %%writefile output.txt

In [16]:
#generates file
with open("output.txt", "w") as f:
    f.write("param money := " + str(money) + ";")
    f.write("\n")
    f.write("param increment := " + str(increment) + ";")
    f.write("\n")
    f.write("param limit := " + str(limit) + ";")
    f.write("\n")
    
    f.write("set States := " )
    f.write("\n")
    for i in state.index:
        f.write(state[state.columns[0]][i])
        f.write("\n")
    f.write(";")
    f.write("\n")
    
    f.write("param entry_cost:=")
    f.write("\n")
    for i in state.index:
        f.write(state[state.columns[0]][i] + "     " + str(state[state.columns[1]][i]))
        f.write("\n")
    f.write(";")
    f.write("\n")
    
    f.write("set Counties := " )
    f.write("\n")
    for i in county.index:
        f.write(county[county.columns[0]][i])
        f.write("\n")
    f.write(";")
    f.write("\n")
    
    for col_name in county.columns:
        if col_name != county.columns[0]:
            f.write("param " + col_name + ":=")
            for i in county.index:
                f.write(county[county.columns[0]][i] + "     " + str(county[col_name][i]))
                .
            
    f.write("param Pairs: ")
    for i in state.index:
        f.write(state[state.columns[0]][i] + " ")
    f.write(":=")
    f.write("\n")
    for i in pairs.index:
        for col in pairs.columns:
            f.write(str(pairs[col][i]))
            f.write(" ")
        f.write("\n")
    f.write(";")

# Determine Basline

## Baseline Using Greedy Choice

Basline represents an upperbound on what can be achieved by greedily selecting counties and speding an equal amount for each. Doesn't use state lines and instead subtracts a fixed amount to account for state entry fees. This results in baselines being overestimations of what is actually achievable if p is chosen to be greater than what the model uses for state costs (recommended).

In [17]:
#sample benefit function that should match whats being used in the .mod file
def reduc(c1,c2,c3,i):
    return c3 * 1/(1+c1*(1.001)**(-c2*i))

In [141]:
p = 10_000_000 #just an estimation
C = [10_000, 50_000, 100_000, 250_000, 500_000, 750_000, 1_000_000, 1_250_000, 1_500_000]

In [142]:
for c in C:
    n= int(np.ceil((money - p) / c))
    new_col = []
    for i in range(baseline.shape[0]):
         new_col.append(reduc(county['Char_1'][i],county['Char_2'][i],county['Char_3'][i],c))
    baseline[str(c)]=new_col
    print(f'Upperbound for spending {c} and n = {n}: ', sum(baseline[str(c)].sort_values(ascending=False)[0:n]))

Upperbound for spending 10000 and n = 9000:  1419909.6953648177
Upperbound for spending 50000 and n = 1800:  1465515.1594352499
Upperbound for spending 100000 and n = 900:  1362826.322207126
Upperbound for spending 250000 and n = 360:  2374836.44277042
Upperbound for spending 500000 and n = 180:  9524102.741457276
Upperbound for spending 750000 and n = 120:  17326435.222843345
Upperbound for spending 1000000 and n = 90:  16442855.921010721
Upperbound for spending 1250000 and n = 72:  13816869.75065875
Upperbound for spending 1500000 and n = 60:  11700367.228653194


## More Accurate Baseline

Creates a more accurate baseline using the average of the above calculations to rank each state by importance. Chooses how many states to consider paying their entry cost and how many counties to consider per state chosen then chooses amount to spend in each county based on remaining money. Does not constrain maximum or minimum number of counties per state ie if 10 states are chosen and 20 counties per state -> chooses 10*20 = 200 best counties overall. Some error due to integer division, overestimation because ceiling function was used. Uses gridsearch-esque method of finding optimal state and county number

In [88]:
entry = state
entry.index = state[state.columns[0]]

In [64]:
new_col = []
for i in range(baseline.shape[0]):
    values = []
    for c in C:
        values.append(baseline[str(c)][i])
    new_col.append(np.mean(values))
baseline["Average"] = new_col

In [129]:
n_states = [5,10,15,20,25,30,35,40,45,50]
n_counties = [10,25,50,75,100]

In [131]:
best = None

for s in tqdm_notebook(n_states):
    M = money
    S = baseline.groupby(by='State').mean()['Average'].sort_values(ascending=False)[0:s]
    for s_name in S.index:  
        M = M - entry['Entry Cost'][s_name]
    for c in n_counties:
        spend = np.ceil(M / s / c)
        C = baseline[baseline['State'].isin(list(S.index))]
        new_col = []
        for i in C.index:
            new_col.append(reduc(county['Char_1'][i],county['Char_2'][i],county['Char_3'][i],spend))
        C[str(spend)] = new_col
        base = sum(C[str(spend)].sort_values(ascending=False)[0:c*s])
        #print(f'Baseline for spending exactly {spend} in {c} counties over {s} available states: {base}')
        
        if best == None:
            best = (base, s, c, spend)
        elif best[0] < base:
            best = (base, s, c, spend)
print('') 
print(f'Best baseline was acheived by spending {best[3]} in {best[2]} counties and {best[1]} states, which yielded {best[0]}')

C:\Users\steph\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  




Best baseline was acheived by spending 971100.0 in 10 counties and 10 states, which yielded 13472299.52260408
